In [3]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
import pandas as pd
import numpy as np
import loompy as lp
#import crick
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import h5py
import tdigest
from tdigest import TDigest
import pickle

/home/vmottaqi/.conda/envs/mcf7/lib/python3.11/site-packages/loompy/bus_file.py:67: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/vmottaqi/.conda/envs/mcf7/lib/python3.11/site-packages/loompy/bus_file.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/vmottaqi/.conda/envs/mcf7/lib/python3.11/site-packages/loompy/bus_file.py:101: NumbaDepr

# 1. Checking the original CSV

## head, shape and subset printing

In [ ]:
df = pd.read_pickle("original.csv")

rows = df.shape[0]
columns = df.shape[1] 
print(f"rows = {rows}")
print(f"columns = {columns}")
print()

print("columns:")
print(df.columns[:17])

df.head()

subset = df.iloc[:3, 0:17]
print(subset)

## Checking unique values of cols

In [ ]:
df = pd.read_pickle("A375_SMILES_siginfo.pkl")

columns = df.columns[:18]

# Count
columns_unique = df[columns].nunique()
# The nunique() function in pandas is used to count the number of unique values in a pandas Series or DataFrame. 

# Create a file to save the output
output_file = "1_nunique.txt"
with open(output_file, "w") as file:
    for name, count in columns_unique.items():
        output = f"{name}: {count} unique values"
        file.write(output + "\n")

## Printing idose and itime values

In [ ]:
df = pd.read_pickle("A375_SMILES_siginfo.pkl")

selected_column_dose = df["pert_idose"]
# Get unique row values in the column
unique_dose = selected_column_dose.unique()

selected_column_time = df["pert_itime"]
unique_time = selected_column_time.unique()

# Create a new TXT file and write unique values
print(f"Unique idose: {unique_dose}")
print()
print(f"Unique time: {unique_time}")

## Pert id and name differences

In [ ]:
df = pd.read_pickle("A375_SMILES_siginfo.pkl")

unique_values_x = df['pert_id'].unique()

# Filter the DataFrame based on unique values in column 'x'
filtered_df = df[df['pert_id'].isin(unique_values_x)]

# Select only columns 'x' and 'y' from the filtered DataFrame
filtered_df = filtered_df[['pert_id', 'pert_name']]

# Print the resulting DataFrame
filtered_df.to_csv('2.csv', index=False)

## Number of zeros and NaNs in the dataset

In [ ]:
df = pd.read_csv("dataset.csv")

total_cells = df.size

# Count the number of cells with zero
zero_cells = np.count_nonzero(df == 0)

# Count the number of cells with NaN values
nan_cells = df.isnull().sum().sum()

# Print the counts
print("Total number of cells:", total_cells)
print("Number of cells with zero:", zero_cells)
print("Number of cells with NaN values:", nan_cells)

## Checking for duplicate cols (genes) and rows (samples)

In [ ]:
# if this works, rm the next two blocks

df = pd.read_csv('original.csv')

# Check for duplicate columns
duplicate_cols = df.columns[df.T.duplicated()].tolist()
duplicate_cols_indices = [df.columns.get_loc(col) for col in duplicate_cols]

# Check for duplicate rows
duplicate_rows = df[df.duplicated()].index.tolist()

# Write the results to a text file
with open('3_duplicates.txt', 'w') as file:
    file.write('duplicate columns (genes):\n')
    file.write(', '.join(duplicate_cols) + '\n')
    file.write(', '.join(map(int, duplicate_cols_indices)) + '\n')
    file.write(f'len: {len(duplicate_cols)}\n\n')
    
    file.write('duplicate rows (samples):\n')
    file.write(', '.join([f'row{i}' for i in duplicate_rows]) + '\n')
    file.write(', '.join(map(str, duplicate_rows)) + '\n')
    file.write(f'len: {len(duplicate_rows)}')

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Check for duplicate names in the columns
duplicates_in_columns = df.columns.duplicated()

# Get the names of the duplicate columns, if any
duplicate_column_names = df.columns[duplicates_in_columns]

if len(duplicate_column_names) > 0:
    print("Duplicate column names found:")
    print(duplicate_column_names)
else:
    print("No duplicate column names found.")

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Initialize a list to store the names of columns with only one unique value
single_value_columns = []

# Iterate over each column and check the number of unique values
for column in df.columns:
    if df[column].nunique() == 1:
        single_value_columns.append(column)

with open('15_unique_genes.txt', 'w') as f:
    for column in single_value_columns:
        value = df.loc[1, column]  # get the second-row value (indexing starts from 0)
        f.write(f"{column}: {value}\n")
                
print(single_value_columns)
print(len(single_value_columns))

In [ ]:
# Not neccessary
df = pd.read_csv('5_sorted_head.csv')

unique_values_count = df["MSRA_cellline"].nunique()
unique_values_count0 = df["ANKRD10_cellline"].nunique()

print(f"Number of unique values in column 'MSRA_cellline': {unique_values_count}")
print(f"Number of unique values in column 'ANKRD10_cellline': {unique_values_count0}")

In [ ]:
# Not neccessary
df = pd.read_csv('5_sorted_head.csv')
print(df["A4GNT"])
print(df["ATP5G2P3"])
print(df["HCRTR2"])

## Histogram of genes (cols) with one unique value

In [ ]:
# Code is written in second part (cell )

## Checking for string values in cols(genes)

In [ ]:
df = pd.read_csv('original.csv')

# Extract the cell attributes from columns 0 to 22 (including)
cell_attributes_df = df.iloc[:, :16]

# Extract the gene expression values from columns 23 onwards
gene_expression_df = df.iloc[:, 16:]
gene_expression_df = gene_expression_df.replace([np.nan, np.inf, -np.inf], -1)

gene_expression_data = gene_expression_df.values.T
#gene_expression_data = gene_expression_data.astype(float)

gene_expression_list = gene_expression_data.tolist()

# Iterate through the list and print the string values
for row in gene_expression_list:
    for value in row:
        if isinstance(value, str):
            print(value)

## Getting loc of a string col

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Find the index of the column "cpd"
cpd_index = df.columns.get_loc("cid")

print(cpd_index)

## 978 landmark genes check difference

In [ ]:
with open('978_L1000.txt', 'r') as txt_file:
    txt_lines = txt_file.readlines()

# Extract values from the second column
txt_column_2 = [line.split()[1] for line in txt_lines]
#print(txt_column_2)

df = pd.read_csv('5_sorted_head.csv')
csv_column_names = df.columns[16:]

# Compare the names from the txt file with the csv column names
overlap_names = set(csv_column_names) & set(txt_column_2)

non_overlap_names = set(csv_column_names) - overlap_names
#non_overlap_l1000 = [elem for elem in set(txt_column_2) if elem not in set(csv_column_names)]
not_overlap = set(txt_column_2) - overlap_names

print("Number of overlapping names:", len(overlap_names))
print("Number of non-overlapping names:", len(non_overlap_names))
print(non_overlap_names)
print("L1000 genes not in the dataset:")
print(not_overlap)

## Indices of columns of interest

In [ ]:
# Not neccessary

df = pd.read_csv("5_sorted_head.csv", engine="python")

column_names = ['pert_iname', 'pert_idose', 'pert_itime']
# Get the indices of the columns
column_indices = [df.columns.get_loc(col_name) for col_name in column_names]

print(column_indices)

# Sorting dataset for replicates

## pkl to csv (sorting operation) (only if input is pkl)

In [ ]:
df = pd.read_pickle("A375_SMILES_siginfo.pkl")

df['pert_idose'] = df['pert_idose'].str.replace(' um', '').astype('float64')
df['pert_itime'] = df['pert_itime'].str.replace(' h', '').astype('float64')

# Sort the dataset by multiple columns
sorted_df = df.sort_values(by=['pert_id', 'pert_idose', 'pert_itime'])

# Reset the index with a new column
sorted_df.reset_index(drop=True, inplace=True)

# Save the sorted data to a new CSV file
sorted_df.to_csv('3_sorted_dataset.csv', index=False)

## Sorting original csv file

In [ ]:
df = pd.read_csv("original.csv")

df['pert_idose'] = df['pert_idose'].str.replace(' um', '').astype('float64')
df['pert_itime'] = df['pert_itime'].str.replace(' h', '').astype('float64')

# Sort the dataset by multiple columns
sorted_df = df.sort_values(by=['pert_id', 'pert_idose', 'pert_itime'])

# Reset the index with a new column
sorted_df.reset_index(drop=True, inplace=True)

# Save the sorted data to a new CSV file
sorted_df.to_csv('3_sorted_dataset.csv', index=False)

## Printing dataset characteristics & making subset

In [ ]:
df = pd.read_csv("4_sorted_dataset.csv")

num_rows = df.shape[0]
num_columns = df.shape[1]

print("columns:")
print(df.columns[:17])
print()
subset = df.iloc[:3, 0:17]
print(subset)

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

first_200_rows = df.head(200)
first_200_rows.to_csv('5_sorted_head.csv', index=False)

## Indices of replicates

In [ ]:
import csv

# Initialize variables
current_val = None
indices = []
output_lines = []

# Open the CSV file for reading
with open("3_sorted_dataset.csv", "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)  # Create a CSV reader

    for index, row in enumerate(csv_reader):
        pert_iname = row["pert_iname"]
        pert_idose = float(row["pert_idose"])
        pert_itime = float(row["pert_itime"])

        if (pert_iname, pert_idose, pert_itime) == current_val:
            indices.append(str(index))  # Convert index to string and append
        else:
            if current_val is not None:
                output_lines.append(",".join(indices))

            current_val = (pert_iname, pert_idose, pert_itime)
            indices = [str(index)]

    if indices:
        output_lines.append(",".join(indices))

# Write the output to a file
with open('8_sorted_indices.txt', 'w') as file:
    for line in output_lines:
        file.write(f"{line}\n")
    
    num_lines = sum(1 for line in file)

print("Number of replicates:", num_lines)

In [ ]:
indices = []

with open('8_sorted_indices.txt', 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespace
        line_list = line.split(',')  # Split the line into a list using comma as the delimiter
        line_list = [int(item) for item in line_list]
        indices.append(line_list)

print(indices)


## Checking max and second-max in replicates

In [ ]:
# Read the text file and split lines by commas
with open("8_sorted_indices.txt", "r") as file:
    lines = file.readlines()

# Convert the text values to integers
data = [[int(num) for num in line.strip().split(",")] for line in lines]

# Find the maximum count of numbers in a row
max_count = max(len(row) for row in data)

# Calculate the lengths of each row
row_lengths = [len(row) for row in data]
# Sort the row_lengths in descending order
row_lengths.sort(reverse=True)
second_max = row_lengths[1]

print("Maximum count of numbers in a replicate:", max_count)
print(f"Second max count of numbers in a replicate: {second_max}")


### Getting description of max

In [ ]:
max_count = 0
max_indices = []

with open("8_sorted_indices.txt", "r") as f:
    for line in f:
        numbers = list(map(int, line.strip().split(',')))
        if len(numbers) > max_count:
            max_count = len(numbers)
            max_indices = numbers

# Read the csv file using pandas
df = pd.read_csv("3_sorted_dataset.csv")

# Use the indices to get the rows and the first 15 columns
selected_rows = df.iloc[max_indices, :15]

# Save the result to a txt file
with open("8.1_max_replicate_des.txt", "w") as f:
    # Write the header
    f.write("indice, " + ', '.join(selected_rows.columns) + "\n")

    for index, row in selected_rows.iterrows():
        f.write(str(index) + ", " + ', '.join(map(str, row)) + "\n")


### Sorting the max replicate with sig_id

In [ ]:
input_file = '3_sorted_dataset.csv'
output_file = '8.5_replicate.csv'

# The row indices to select
start_row = 21378
end_row = 22016

# Read specific rows from the CSV file into a DataFrame
df_selected_rows = pd.read_csv(input_file, skiprows=range(1, start_row), nrows=(end_row - start_row + 1))

unique_count = df_selected_rows['sig_id'].nunique()

print(f"The number of unique values in the 'sig_id' column is {unique_count}.")


### Changing the indices file based on the new criterion (to be written)

# Pearson Correlation

## small PC test

In [ ]:
df = pd.read_pickle("4_sorted_head.pkl")

# Select the columns of interest (from column 23 onwards)
gene_columns = df.columns[16:]

#replicates = df.loc[:4, gene_columns].apply(pd.to_numeric, errors='coerce')
df1 = df.loc[:3,gene_columns].apply(pd.to_numeric, errors='coerce').dropna(axis=1)

pairwise_correlation = np.corrcoef(df1, rowvar=True)
print(pairwise_correlation)

## Medium PC test

In [ ]:
# NOT Necessary

df = pd.read_pickle("4_sorted_dataset.pkl")

# Select the columns of interest (from column 23 onwards)
gene_columns = df.columns[16:]
df1 = df.loc[:, gene_columns].apply(pd.to_numeric, errors='coerce')

def pairwise(vector):
    for i in range(len(vector) - 1):
        for j in range(i+1, len(vector)):
            # Get the two rows for comparison
            row1 = df1.loc[vector[i], gene_columns].astype(float)
            row2 = df1.loc[vector[j], gene_columns].astype(float)

            # Drop columns with NaN values from both rows
            non_nan_columns = row1.notna() & row2.notna()
            row1_dropped = row1[non_nan_columns]
            row2_dropped = row2[non_nan_columns]

            # Compute the correlation coefficient for the non-NaN values
            correlation = round(np.corrcoef(row1_dropped, row2_dropped)[0, 1], 1)

            # Store the correlation coefficient in the matrix
            pairwise_correlation[vector[j], vector[i]] = correlation
            
# Initialize an empty correlation matrix
pairwise_correlation = np.full((15, 15), 33.00)

indices = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        
for elem in indices:
    pairwise(elem)

# Save the correlation matrix to a file
np.savetxt('9_test_pairwise.csv', pairwise_correlation, delimiter=',')

## Pairwise Pearson CORR operation

In [ ]:
import pandas as pd
import numpy as np
import h5py

def pairwise(vector, matrix, chunk_df):
    for i in range(len(vector) - 1):
        for j in range(i + 1, len(vector)):
            row1 = chunk_df.loc[vector[i], :]
            row2 = chunk_df.loc[vector[j], :]
            non_nan_columns = row1.notna() & row2.notna()
            row1_dropped = row1[non_nan_columns]
            row2_dropped = row2[non_nan_columns]
            correlation = round(np.corrcoef(row1_dropped, row2_dropped)[0, 1], 3)
            matrix[vector[j], vector[i]] = correlation

indices = []
with open('8_sorted_indices.txt', 'r') as file:
    for line in file:
        line = line.strip()
        line_list = line.split(',')
        line_list = [int(item) for item in line_list]
        indices.append(line_list)

chunksize = 3000
pairwise_correlation = np.zeros((31680, 31680))
prev_chunk_df = None

# Read the CSV file chunk-by-chunk
for start_idx in range(0, 31680, chunksize):
    chunk = pd.read_csv("3_sorted_dataset.csv", skiprows=range(1, start_idx + 1), nrows=chunksize)
    actual_rows = chunk.shape[0]  # Get the actual number of rows in the chunk
    end_idx = start_idx + actual_rows - 1  # Adjust the end index based on the actual number of rows
    
    gene_columns = chunk.columns[16:]
    chunk_df = chunk.loc[:, gene_columns].apply(pd.to_numeric, errors='coerce')
    chunk_df.index = range(start_idx, end_idx + 1)  # Set the index based on the actual number of rows

    
    # Combine with previous chunk if it exists
    if prev_chunk_df is not None:
        combined_df = pd.concat([prev_chunk_df, chunk_df])
    else:
        combined_df = chunk_df

    # Find the groups that overlap with this chunk's index range
    relevant_indices = [grp for grp in indices if any(start_idx <= x <= end_idx for x in grp)]
    
    for elem in relevant_indices:
        if all(x in combined_df.index for x in elem):
            pairwise(elem, pairwise_correlation, combined_df)
    
    prev_chunk_df = chunk_df

# Save the resulting matrix
with h5py.File('10_pairwise_correlation.h5', 'w') as hf:
    hf.create_dataset('pairwise_correlation', data=pairwise_correlation)

## test h5 file

In [ ]:
with h5py.File('10_pairwise_correlation.h5', 'r') as hf:
    pairwise_correlation = hf['pairwise_correlation'][:8,:8]

print(pairwise_correlation.shape[0])
print()
print(pairwise_correlation)

# Assessing PC h5 file

## Checking values in matrix rows and cols

In [ ]:
with h5py.File('10_pairwise_correlation.h5', 'r') as hf:
    dataset = hf['pairwise_correlation'][:]

index = 0
    
# Count the number of non-zero values in the selected row
row_i = dataset[index, :]
col_i = dataset[:, index]


count_row = np.count_nonzero(row_i)
count_column = np.count_nonzero(col_i)


# Count the number of values under 0.85 in row i
num_values_under_085_row = np.count_nonzero((row_i < 0.99) & (row_i != 0))
num_values_under_085_col = np.count_nonzero((col_i < 0.99) & (col_i != 0))

#print(f"Length: {len(num_values)}")
print("Row:")
print(f"Non-zero in row {index}: {count_row}")
print(f"Non-zero in column {index}: {count_column}")
print()

print(f"Under threshold in row: {num_values_under_085_row}")
print(f"Under threshold in column: {num_values_under_085_col}")

In [ ]:
# Not neccessary

with h5py.File('10_pairwise_correlation.h5', 'r') as hf:
    dataset = hf['pairwise_correlation'][:]

index = 0
    
# Count the number of non-zero values in the selected row
row_i = dataset[index, :]
col_i = dataset[:, index]


selected_values = []

for i, j in zip(row_i, col_i):
    # Check if both values are non-zero and less than 0.99
    if i != 0 and i < 0.99:
        selected_values.append(i)
    if j != 0 and j < 0.99:
        selected_values.append(j)

count_under_085 = sum(1 for value in selected_values if value < 0.95)

print(f"All values in row {index}: {len(selected_values)}")
print(f"Values under threshold: {count_under_085}")

## Double checking the PC value in csv (two rows)

In [ ]:
# Also for comparison, you can check "small PC test" section (upper)

In [ ]:
df = pd.read_csv("4_sorted_head.csv")

# Select the columns of interest (from column 23 onwards)
gene_columns = df.columns[16:]
df1 = df.loc[:, gene_columns].apply(pd.to_numeric, errors='coerce')


row1 = df1.loc[0, gene_columns].astype(float)
row2 = df1.loc[1, gene_columns].astype(float)


non_nan_columns = row1.notna() & row2.notna()
row1_dropped = row1[non_nan_columns]
row2_dropped = row2[non_nan_columns]


mean_row1 = row1_dropped.mean()
mean_row2 = row2_dropped.mean()


row1_minus_mean = row1_dropped - mean_row1
row2_minus_mean = row2_dropped - mean_row2

sum_of_products_deviations = (row1_minus_mean * row2_minus_mean).sum()

std_row1 = row1_dropped.std()
std_row2 = row2_dropped.std()

PC0 = sum_of_products_deviations / (std_row1 * std_row2) 
PC = PC0 / len(row2_dropped)
    
print(f"PC between Row 1 and Row 2: {PC}")

## Histogram of pairwise PC 

In [1]:
with h5py.File('10_pairwise_correlation.h5', 'r') as hf:
    pairwise_correlation = hf['pairwise_correlation'][:]

# Flatten the matrix into a 1D array
values = pairwise_correlation.flatten()

# Remove zeros from the array
non_zero_values = values[values != 0]

# Create a histogram of the non-zero values
plt.hist(non_zero_values, bins=80, ec="purple")
plt.xlabel('Pearson Coefficients')
plt.ylabel('Count')
plt.title('Pairwise Pearson correlation among replicates (Sigcom A375)')
#plt.savefig('11_histogram_A375_before.png', dpi=400)
plt.show()

NameError: name 'h5py' is not defined

# H5 modification (indices)

## Adding indices to h5

In [ ]:
# Execute separately on the server

with h5py.File('10_pairwise_correlation.h5', 'r') as hf:
    # Get the dataset (assuming your dataset is named 'correlation')
    dataset = hf['pairwise_correlation'][:]
    
    # Get the shape of the dataset
    num_rows, num_columns = dataset.shape
    
    # Create a new array containing the indices of the columns
    column_indices = np.arange(num_columns, dtype=int)
    
    # Append the new row to the matrix
    dataset_with_indices = np.vstack((dataset, column_indices))
    
# Save the updated matrix with indices to the h5 file
with h5py.File('13_pairwise_correlation_indices.h5', 'w') as hf:
    hf.create_dataset('pairwise_correlation', data=dataset_with_indices)


## How many values below threshold? & checking last line

In [ ]:
# Double check with histogram too!

In [ ]:
count = 0

with h5py.File("13_pairwise_correlation_indices.h5", "r") as hf:
    dset = hf["pairwise_correlation"]
    last_row = dset[-1,:]
    rows, cols = dset.shape
    
    # Define chunk size (you may need to adjust this based on your memory)
    chunk_rows = 5000
    threshold = 0.92
    
    for start_row in range(0, rows, chunk_rows):
        end_row = min(rows, start_row + chunk_rows)
        
        # Read a chunk of data into memory
        chunk_data = dset[start_row:end_row, :]
        
        # Update the counter
        count += np.sum((chunk_data != 0) & (chunk_data < threshold))

print("last row of h5 file:")
print(last_row, "\n")
print(f"Number of non-zero values below 0.92: {count}")

# H5 below threshold cleaning

## 1. txt creation

In [ ]:
# Find them (the ones with the most num of correlations below the threshold), delete them. Step by step

In [ ]:
with h5py.File('13_pairwise_correlation_indices.h5', 'r') as hf:
    dataset = hf['pairwise_correlation'][:]

selected_samples = []

def count(index, threshold, number):
    row_i = dataset[index, :]
    col_i = dataset[:, index]

    # Select non-zero values for row_i and col_i
    row_values = row_i[row_i != 0]
    col_values = col_i[col_i != 0]
    
    # Concatenate the selected non-zero values
    selected_values = np.concatenate([row_values, col_values])

    count_under_threshold = np.count_nonzero(selected_values < threshold)
    if count_under_threshold >= number:
        selected_samples.append(index)

for i in range(dataset.shape[0]-1):
    count(i, 0.9, 1)

print(len(selected_samples))
print(selected_samples)
np.savetxt("14.txt", selected_samples, fmt='%d')

#selected_samples = np.loadtxt("selected_samples.txt", dtype=int)

## 2. removing samples in the matrix

In [ ]:
# Step 1: Read the indices from the text file into a list
with open("14.txt", "r") as file:
    selected_samples = [int(line.strip()) for line in file]

# Step 2: Open the h5 file in read-write mode
with h5py.File("40.h5", "r") as hf:
    # Step 3: Remove the rows and columns from the dataset
    correlation_matrix = hf['pairwise_correlation'][:]
    filtered_correlation_matrix0 = np.delete(correlation_matrix, selected_samples, axis=0)
    filtered_correlation_matrix = np.delete(filtered_correlation_matrix0, selected_samples, axis=1)

with h5py.File('41_clean_dataset.h5', 'w') as hf:
    hf.create_dataset('pairwise_correlation', data=filtered_correlation_matrix)

## 3. Checkinf rows and cols of h5

In [ ]:
with h5py.File('13_pairwise_correlation_indices.h5', 'r') as hf:
    correlation_matrix = hf['pairwise_correlation'][:]

print(correlation_matrix.shape)
print(correlation_matrix)

## 4. New Histogram

In [ ]:
with h5py.File('50_Cleaned_Data_indices.h5', 'r') as hf:
    pairwise_correlation = hf['correlation'][:]

# Flatten the matrix into a 1D array
values = pairwise_correlation.flatten()

# Remove zeros from the array
non_zero_values = values[values != 0]

# Create a histogram of the non-zero values
plt.hist(non_zero_values, bins=100, ec="purple")
plt.xlim(0, 1)
plt.xlabel('Pearson Coefficients')
#plt.ylabel('Count')
plt.title('Pairwise Pearson correlation among replicates_cleaned (MCF7)')
#plt.savefig('42_histogram_clean_MCF7.png', dpi=400)
plt.show()

# Cleaning original CSV file

## 1. Cleaning

In [ ]:
with h5py.File('50_Cleaned_Data_indices.h5', 'r') as hf:
    
    # Assuming your dataset is named 'matrix', get the last row
    indices_to_keep = hf['correlation'][-1,:]
    
df = pd.read_csv('9_sorted_dataset.csv')

# Filter the DataFrame to keep only the rows with the specified indices
filtered_df = df[df.index.isin(indices_to_keep)]

# Add a new column with the old indices
filtered_df.insert(0, 'old_index', filtered_df.index)

# Reset the index of the DataFrame
filtered_df.reset_index(drop=True, inplace=True)

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('51_MCF7_clean.csv', index=False)

## 2. Checking rows and cols

In [ ]:
df = pd.read_csv('51_MCF7_clean.csv')

print(df.shape)
#print(f"Number of rows in clean csv file: {len(df)}")

## 3. Making clean_head file

In [ ]:
df = pd.read_csv('clean_dataset.csv')
filtered_df = df.head(200)
filtered_df.to_csv('52_head.csv', index=False)

In [ ]:
# Not necessary
df = pd.read_csv('5_sorted_head.csv')
print(df["A4GNT"])
print(df["ATP5G2P3"])
print(df["HCRTR2"])

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Extract the gene expression values from columns 23 onwards
gene_expression_df = df.iloc[:, 16:]

# Get the number of columns from 23 onward
num_columns_from_23 = gene_expression_df.shape[1]

# Get the unique gene names from column 23 onward
unique_gene_names = gene_expression_df.columns.unique()
num_unique_gene_names = len(unique_gene_names)

print(f"Number of columns from column 16 onward: {num_columns_from_23}")
print(f"Number of unique gene names: {num_unique_gene_names}")

## 4. Printing last 50 cols (genes)

In [ ]:
df = pd.read_csv('5_sorted_head.csv')
print(df.columns[-30:])

## 5. Getting loc of a specific col (gene)

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Find the index of the column "cpd"
cpd_index = df.columns.get_loc("cid")

print(cpd_index)